## importing library

In [20]:
from datetime import datetime
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
import numpy as np
import string
import preprocessor as p
import os
import time

import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import snscrape.modules.twitter as sntwitter


# Setting up Tokens from Twitter Developer Accounts
#### You can apply for Twitter Developer account and get your own keys

In [6]:

# consumer_key = <use-your-own-key>
# consumer_secret = <use-your-own-key>
# access_token = <use-your-own-key>
# access_token_secret = <use-your-own-key>

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


# Scraping Tweets from Official Fertile Ground Account
#### The data is scraped and exported to 'fertilegroundtweets.csv' file in data folder (Make sure you have created the data folder in your current directory. And you can always change the name of the file as you want.
You can change the username to scrape data from any public twitter profile.
#### Right now, the code to create csv files from the data is commented out, we can uncomment them by removing the hashtags in the beginning of the line of the code

In [8]:
username = '_fertileground_'
count = 1000
try:    
    tweets = tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(count)
    tweetList = [[tweet.created_at, tweet.id, tweet.full_text, tweet.user.location]
                   for tweet in tweets]
  
    tweets_df = pd.DataFrame(tweetList)
except BaseException as e:
    print('failed,', str(e))
    time.sleep(4)

tweets_df.columns = ['Tweet Date', 'Tweet ID', 'Tweet text', 'Tweet Location']
print(tweets_df)

#Saving data into fertilegroundtweets.csv file
tweets_df.to_csv('data/fertilegroundtweets.csv')

             Tweet Date             Tweet ID  \
0   2021-09-28 16:11:10  1442884599319724034   
1   2021-09-28 14:45:16  1442862983219007493   
2   2021-09-28 14:06:12  1442853150260662286   
3   2021-09-28 13:54:16  1442850148313350147   
4   2021-09-27 18:48:53  1442561899963559945   
..                  ...                  ...   
581 2019-06-19 21:08:20  1141452680130420738   
582 2019-06-19 21:04:46  1141451780070477826   
583 2019-06-19 20:49:10  1141447855363829760   
584 2019-03-21 21:07:00  1108837436958953473   
585 2019-02-02 16:49:42  1091740454146723841   

                                            Tweet text Tweet Location  
0    In November 2018, the City of Jackson was sele...    Jackson, MS  
1    We are so excited about the schedule for our F...    Jackson, MS  
2    In November 2018, the City of Jackson was sele...    Jackson, MS  
3    Check out our very own Chef @ey_williams of Fa...    Jackson, MS  
4    Thank you MyCityJXN for featuring the Fertile ...    Jacks

# Scraping Tweets using snscrape for different terms and exporting one single CSV file with all the tweets 
#### We used keywords such as terms=['fertile ground Jackson Mississippi', 'food access Jackson Mississippi', 'food insecurity Jackson Mississippi', 'food security Jackson Mississippi', 'food health Jackson Mississippi', 'food deserts Jackson Mississippi', 'food swamps Jackson Mississippi','#fertilegroundjxn','Jackson Mississippi #healthyfood'] to scrape the data
#### Other parameters that we used for the scrape 
#### geocode = '32.2998,-90.1848'
#### radius = '50km'
#### date = "2018-03-01"
##### The data is scraped and exported to 'tweetsallCounts.csv' file

In [21]:
# for different search terms and to get the reply count, retweet counts and other data from the tweets related to the search term.

tweetList = []

terms=['fertile ground Jackson Mississippi', 'food access Jackson Mississippi', 'food insecurity Jackson Mississippi', 'food security Jackson Mississippi', 'food health Jackson Mississippi', 'food deserts Jackson Mississippi', 'food swamps Jackson Mississippi','#fertilegroundjxn','Jackson Mississippi #healthyfood']

geocode = '32.2998,-90.1848'
radius = '50km'
date = "2018-03-01"

for term in terms:
    #for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' geocode:'+geocode+','+radius+' since:'+date).get_items()):      
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' since:2018-03-01').get_items()):      
        if i>1000:
            break
        tweetList.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])

tweets_df = pd.DataFrame(tweetList, columns=['Datetime', 'Tweet Id', 'Text', 'Username','User Location','Reply Count','Retweet Count','Like Count','Quote Count'])

tweets_df.drop_duplicates(subset="Tweet Id", keep=False, inplace=True)

#Exporting data to tweetsallCounts.csv file
tweets_df.to_csv("data/tweetsallCounts.csv")
tweets_df

,Datetime,Tweet Id,Text,Username,User Location,Reply Count,Retweet Count,Like Count,Quote Count
0,2020-12-27 22:08:13+00:00,1343317791483559937,"‘Fertile Ground’ Explores How Jackson, Mississ...",komala_hayes,,0,1,2,0
1,2020-07-13 05:38:10+00:00,1282549864392425472,"‘Fertile Ground’ Explores How Jackson, Mississ...",DisbldUrbanFarm,"Kalamazoo, MI",0,0,0,0
2,2020-07-12 17:28:00+00:00,1282366111032455168,"""'Fertile Ground' Explores How Jackson, Missis...",hollowearths,"Queens, NY",0,0,4,0
4,2020-06-22 12:10:20+00:00,1275038407677415427,"‘Fertile Ground’ Explores How Jackson, Mississ...",DavidDuguid27,,0,0,0,0
5,2020-06-19 14:35:58+00:00,1273987896450732032,Looking forward to watching Fertile Ground abo...,alicetahi,"Cardiff, Wales",0,0,2,0
...,...,...,...,...,...,...,...,...,...
90,2018-11-30 03:19:43+00:00,1068343793320112128,"Bloomberg Philanthropies has named Jackson, MS...",reflectionchef,"Jackson, Mississippi",1,3,9,1
91,2018-11-29 20:27:21+00:00,1068240018433601537,We’re grateful to @BloombergDotOrg for this op...,ChokweALumumba,"Jackson, MS",4,21,81,6
92,2018-11-29 19:18:49+00:00,1068222771199729665,#JacksonMS will receive the Bloomberg Philanth...,NicLott,"Jackson, MS",0,0,1,1
93,2018-11-29 18:15:53+00:00,1068206932744392705,Big news for @CityofJacksonMS: the city has wo...,MSarts,"Jackson, MS",2,9,26,4


# Scraping using snscrape for each terms individually and exporting to CSV files

### In this we scrape data like above but separately for individual keywords.   

In [ ]:
#for individual different search terms and to get the reply count, retweet counts and other data from the tweets related to the search term.


def difkeyword(keyword):
    tweets_list2 = []

    term=keyword
    geocode='32.2998,-90.1848'
    radius='50km'
    date="2018-03-01"

    #for term in terms: 
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' geocode:'+geocode+','+radius+' since:'+date).get_items()):

        if i>500:
            break

        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])




    tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username','User Location','Reply Count','Retweet Count','Like Count','Quote Count'])




    return tweets_df2
    
tweets_df2=difkeyword('fertile ground')
tweets_df2.to_csv('data/tweets_fertileground.csv')

tweets_df2=difkeyword('food access')
tweets_df2.to_csv('data/tweets_foodaccess.csv')

tweets_df2=difkeyword('food security')
tweets_df2.to_csv('data/tweets_foodsecurity.csv')

tweets_df2=difkeyword('food insecurity')
tweets_df2.to_csv('data/tweets_foodinsecurity.csv')

tweets_df2=difkeyword('food swamps')
tweets_df2.to_csv('data/tweets_foodswamps.csv')

tweets_df2=difkeyword('food deserts')
tweets_df2.to_csv('data/tweets_fooddeserts.csv')

tweets_df2=difkeyword('#fertilegroundjxn')
tweets_df2.to_csv('data/tweets_fertilegroundjxn.csv')

tweets_df2=difkeyword('#healthyfood')
tweets_df2.to_csv('data/tweets_healthyfood.csv')

tweets_df2=difkeyword('food health')
tweets_df2.to_csv('data/tweets_foodhealth.csv')

### In this you are prompted for keyword as a user input to scrape data. You can manually change the date, geocode and radius to scrape data
#### The scraped data will be saved in the data folder with the file name as "keyword".csv (for ex- keyword=food, then file name will be food.csv

In [ ]:
def difkeyword(keyword):
    tweets_list2 = []

    term=keyword
    geocode='32.2998,-90.1848'
    radius='50km'
    date="2018-03-01"


   
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' geocode:'+geocode+','+radius+' since:'+date).get_items()):

        if i>1000:
            break

        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])


    tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username','User Location','Reply Count','Retweet Count','Like Count','Quote Count'])



   # tweets_df2.to_csv("tweets_healthyfood.csv")
    return tweets_df2

keyword=input("Enter a keyword to scrape:")
tweets_df=difkeyword(keyword)
tweets_df.to_csv("data/"+keyword+"_scraped.csv")
tweets_df

### Same as above but this just uses date and keyword as search parameters. The date is set as 2018-03-01. You can change that manually too if you would like. This is helpful for hashtag and very specific keywords of which people can tweet from any part of the world

In [17]:
def difkeyword(keyword):
    tweets_list2 = []

    term=keyword   
    date="2018-03-01"


    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' since:2018-03-01').get_items()):  

#Max number of tweets allowed. You can increase or decrease this to get the specific number of data. Currently it is set as 1000.
        if i>1000:
            break

        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])


    tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username','User Location','Reply Count','Retweet Count','Like Count','Quote Count'])



   # tweets_df2.to_csv("tweets_healthyfood.csv")
    return tweets_df2

keyword=input("Enter a keyword to scrape:")
tweets_df=difkeyword(keyword)
tweets_df.to_csv("data/"+keyword+"_scraped.csv")
tweets_df

Enter a keyword to scrape:#fertilegroundjxn


,Datetime,Tweet Id,Text,Username,User Location,Reply Count,Retweet Count,Like Count,Quote Count
0,2021-09-14 16:06:06+00:00,1437809894623748099,We are so excited to announce the schedule for...,_fertileground_,"Jackson, MS",1,1,2,1
1,2020-08-24 17:08:36+00:00,1297943904080232454,”The monster is seemingly in a food deficient ...,_fertileground_,"Jackson, MS",1,0,2,0
2,2020-08-12 19:01:46+00:00,1293623730296496128,Food is the epicenter of family gatherings— th...,_fertileground_,"Jackson, MS",0,1,3,0
3,2020-06-10 20:23:04+00:00,1270813757208514561,The Film | #FertileGroundJXN https://t.co/t1L...,jxntransplants,"Jackson, MS",0,1,2,0
4,2020-04-22 14:08:16+00:00,1252962426770898946,Today marks the 50th anniversary of Earth Day ...,_fertileground_,"Jackson, MS",1,3,4,0
5,2020-04-02 17:22:58+00:00,1245763667548921857,“What happens in South Jackson is what happens...,_fertileground_,"Jackson, MS",1,12,13,0
6,2020-01-17 15:12:28+00:00,1218189350057455618,@JacksonMedMall Her family has been farming in...,_fertileground_,"Jackson, MS",0,0,1,0
7,2020-01-10 14:52:27+00:00,1215647596913491968,@BilalQizilbash The R U Hungry program feeds t...,_fertileground_,"Jackson, MS",0,0,0,0
8,2020-01-08 22:27:11+00:00,1215037257780015104,The blueberries have arrived! 150 blueberry pl...,_fertileground_,"Jackson, MS",0,0,5,0
9,2019-12-16 15:01:12+00:00,1206590103361118208,Enrika Williams @ey_williams is a Jackson chef...,_fertileground_,"Jackson, MS",1,2,8,1
